# Creating Python Virtual Environments with conda

- [Overview](#conda-virual-env-overview)
- [Setting Up a Virtual Environment Using conda](#setting-up-a-virtual-environment-using-conda)
  - [Creating a conda Virtual Environment from a File](#creating-a-conda-environment-from-a-file)
- [Setting Up a RAPIDS conda Environment with cuDF and cuML](#setting-up-a-rapids-conda-environment)

<a id="conda-virual-env-overview"></a>
## Overview

A Python virtual environment is a named, isolated, working copy of Python that maintains its own files, directories, and paths so that you can work with specific versions of libraries or Python itself without affecting other Python projects.
Virtual environments make it easy to cleanly separate projects and avoid problems with different dependencies and version requirements across components.
The conda command-line interface (CLI) is the preferred interface for managing installations and virtual environments with the Anaconda Python distribution.
The Jupyter Notebook service of the Iguazio Data Science Platform ("the platform") comes pre-deployed with conda.
This tutorial explains how to use conda to create a Python virtual environment that will be available as a custom kernel in Jupyter Notebook.
For general information about using conda to create virtual environments, see the [conda documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html).

<a id="setting-up-a-virtual-environment-using-conda"></a>
## Setting Up a Virtual Environment Using conda

Follow these steps from your Jupyter Notebook service to create a Python virtual environment using conda: 

1.  <a id="conda-virt-env-setup-step-create-terminal"></a>Create a new terminal by selecting the **New Launcher** option (`+` icon) from the top action toolbar in the left sidebar, and then selecting **Terminal** from the main work area.<br>
   The next steps should be executed from your new terminal, except where otherwise specified.


2.  <a id="conda-virt-env-setup-step-create-env"></a>Create a new Python virtual environment by running the following command.
    Replace `<environment name>` with your preferred virtual-environment name:
    ```sh
    conda create -n <environment name> ipykernel python=3.6
    ```

    <br>For example, the following command creates an environment named "myenv":<br>
    ```sh
    conda create -n myenv ipykernel python=3.6
    ```


3. <a id="conda-virt-env-setup-step-install-packages"></a>Install the desired Python packages by running the following command.
   Replace `<environment name>` with the name of the environment that you created in the previous step and `<package>` with the name of your selected package, optionally followed by `=<version>`; you can specify multiple packages:
   ```sh
   conda install -n <environment name> <package> [<package> ...]
   ```

   <br>For example, the following command installs the SciPy, pandas version 0.25.0, and TensorFlow version 1.13.1 packages for a "myenv" environment:
   ```sh
   conda install -n myenv scipy pandas=0.25.0 tensorflow=1.13.1 python=3.6
   ```


4.  <a id="conda-virt-env-setup-step-export-env"></a>Export your new virtual environment to an environment file in a platform data container by running the following command.
    Replace `<container name>` with the name of a platform data container, `<directory path>` with an optional relative container-directories path, and `<environment name>` with the name of the environment that you created:
    ```sh
    conda env export -n <environment name> > /v3io/<container name>[/<directory path>]/<environment name>.yaml
    ```

    <br>It's recommended that you save the environment to a virtual-environments directory in your running-user home directory (**users/&lt;running user&gt;**).
    For example, the following command creates a **users/&lt;running user&gt;/virtual_env/myenv.yaml** file:
    ```sh
    conda env export -n myenv > /v3io/users/$V3IO_USERNAME/virtual_env/myenv.yaml
    ```
    You can use the `User` data mount to the running-user directory to shorten this command (see [Platform Data Containers](data-ingestion-and-preparation/README.ipynb#platform-data-containers)):
    ```sh
    conda env -n myenv export > /User/virtual_env/myenv.yaml
    ```


5.  <a id="conda-virt-env-setup-step-refresh-ui"></a>Refresh the JupyterLab UI to apply your changes.
    After refreshing the UI, you should be able to see your new environment in the list of available kernels in Jupyter Notebook.

<a id="creating-a-conda-environment-from-a-file"></a>
### Creating a conda Virtual Environment from a File

If, for any reason, your conda environment variable is removed from Jupyter Notebook, you can easily deploy it again by using the YAML environment file that you exported in [Step 4](#conda-virt-env-setup-step-export-env) of the setup procedure:

1.  Open a new Jupyter terminal.

2.  Run the following command to recreate the environment from the environment file.
    Replace `<directory path>` and `<environment name>` to set the path to the environment file that you saved as part of the initial setup:
    ```sh
    conda env create --file /v3io/<container name>[/<directory path>]/<environment name>.yaml
    ```

    For example, the following command loads a **users/&lt;running user&gt;/virtual_env/myenv.yaml** environment file; the command uses the `User` running-user directory data mount to the running-user directory in the "users" container:
    ```sh
    conda env create --file  /User/virtual_env/myenv.yaml
    ```

<a id="setting-up-a-rapids-conda-environment"></a>
## Setting Up a RAPIDS conda Environment with cuDF and cuML

To use the cuDF and cuML RAPIDS libraries, you need to create a RAPIDS conda environment.
Use the following command to create a RAPIDS conda environment named `rapids`:

```sh
conda create -n rapids -c rapidsai -c nvidia -c anaconda -c conda-forge -c defaults ipykernel rapids=0.14 python=3.7 cudatoolkit=10.1
```